#### Reading the csv files

In [1]:
ames1 = pd.read_csv('../datasets/train.csv')

In [2]:
test_kaggle = pd.read_csv('../datasets/test.csv')

#### Importing everything

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV

%matplotlib inline

#### Some needed code for better visualization

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

<IPython.core.display.Javascript object>

In [5]:
pd.set_option('display.min_rows', 60)  # Changing output display options
pd.set_option('display.max_rows', 600)  # Changing output display options

In [6]:
ames1.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [7]:
test_kaggle.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


#### Joining both dataframes on top of each other to clean them at the same time

In [8]:
ames = pd.concat([ames1.drop(columns=['SalePrice']), test_kaggle], ignore_index=True)

In [9]:
ames.columns = ames.columns.str.replace(' ', '_')

#### Changing the null values per column to the desired values

In [10]:
values = {'Lot_Frontage': 0, 
          'Alley': 'No_Alley_Access', 
          'Mas_Vnr_Type': 'None', 
          'Mas_Vnr_Area': 0, 
          'Bsmt_Qual': 'No_Basement',
          'Bsmt_Cond': 'No_Basement',
          'Bsmt_Exposure': 'No_Basement',
          'BsmtFin_Type_1': 'No_Basement',
          'BsmtFin_SF_1': 0,
          'BsmtFin_Type_2': 'No_Basement',
          'BsmtFin_SF_2': 0,
          'Bsmt_Unf_SF': 0,
          'Total_Bsmt_SF': 0,
          'Bsmt_Full_Bath': 0,
          'Bsmt_Half_Bath': 0,
          'Fireplace_Qu': 'No_Fireplace',
          'Garage_Type': 'No_Garage',
          'Garage_Yr_Blt': 'No_Garage',
          'Garage_Finish': 'No_Garage',
          'Garage_Cars': 0,
          'Garage_Area': 0,
          'Garage_Qual': 'No_Garage',
          'Garage_Cond': 'No_Garage',
          'Pool_QC': 'No_Pool',
          'Fence': 'No_Fence',
          'Misc_Feature': 'None',
          'Electrical': 'Mix'}
ames.fillna(value=values, inplace=True)

In [11]:
ames.isnull().sum().sum()

0

#### Checking some stats

In [12]:
ames.describe()

,Id,PID,MS_SubClass,Lot_Frontage,Lot_Area,Overall_Qual,Overall_Cond,Year_Built,Year_Remod/Add,Mas_Vnr_Area,...,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,3Ssn_Porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Yr_Sold
count,2930.00000,2.930000e+03,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,1465.50000,7.144645e+08,57.387372,57.647782,10147.921843,6.094881,5.563140,1971.356314,1984.266553,101.096928,...,472.658362,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444
std,845.96247,1.887308e+08,42.638025,33.499441,7880.017759,1.411026,1.111537,30.245361,20.860286,178.634545,...,215.187196,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613
min,1.00000,5.263011e+08,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,733.25000,5.284770e+08,20.000000,43.000000,7440.250000,5.000000,5.000000,1954.000000,1965.000000,0.000000,...,320.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,1465.50000,5.354536e+08,50.000000,63.000000,9436.500000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,480.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2197.75000,9.071811e+08,70.000000,78.000000,11555.250000,7.000000,6.000000,2001.000000,2004.000000,162.750000,...,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2930.00000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1488.000000,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


#### Changing categorical values of certain features to numbers in order to get a strongest ordinal correlation

In [13]:
ames['Exter_Qual'].value_counts()

TA    1799
Gd     989
Ex     107
Fa      35
Name: Exter_Qual, dtype: int64

In [14]:
ames['Exter_Qual'] = ames['Exter_Qual'].map({'Ex': 5,
                                             'Gd': 4,
                                             'TA': 3,
                                             'Fa': 2})

In [15]:
ames['Exter_Cond'].value_counts()

TA    2549
Gd     299
Fa      67
Ex      12
Po       3
Name: Exter_Cond, dtype: int64

In [16]:
ames['Exter_Cond'] = ames['Exter_Cond'].map({'Ex': 5,
                                             'Gd': 4,
                                             'TA': 3,
                                             'Fa': 2,
                                             'Po': 1})

In [17]:
ames['Bsmt_Qual'].value_counts()

TA             1283
Gd             1219
Ex              258
Fa               88
No_Basement      80
Po                2
Name: Bsmt_Qual, dtype: int64

In [18]:
ames['Bsmt_Qual'] = ames['Bsmt_Qual'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1,
                                           'No_Basement': 0})

In [19]:
ames['Bsmt_Cond'].value_counts()

TA             2616
Gd              122
Fa              104
No_Basement      80
Po                5
Ex                3
Name: Bsmt_Cond, dtype: int64

In [20]:
ames['Bsmt_Cond'] = ames['Bsmt_Cond'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1,
                                           'No_Basement': 0})

In [21]:
ames['Bsmt_Exposure'].value_counts()

No             1906
Av              418
Gd              284
Mn              239
No_Basement      83
Name: Bsmt_Exposure, dtype: int64

In [22]:
ames['Bsmt_Exposure'] = ames['Bsmt_Exposure'].map({'Gd': 4,
                                           'Av': 3,
                                           'Mn': 2,
                                           'No': 1,
                                           'No_Basement': 0})

In [23]:
ames['Heating_QC'].value_counts()

Ex    1495
TA     864
Gd     476
Fa      92
Po       3
Name: Heating_QC, dtype: int64

In [24]:
ames['Heating_QC'] = ames['Heating_QC'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1})

In [25]:
ames['Kitchen_Qual'].value_counts()

TA    1494
Gd    1160
Ex     205
Fa      70
Po       1
Name: Kitchen_Qual, dtype: int64

In [26]:
ames['Kitchen_Qual'] = ames['Kitchen_Qual'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1})

In [27]:
ames['Functional'].value_counts()

Typ     2728
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sal        2
Sev        2
Name: Functional, dtype: int64

In [28]:
ames['Functional'] = ames['Functional'].map({'Typ': 7,
                                           'Min1': 6,
                                           'Min2': 5,
                                           'Mod': 4,
                                           'Maj1': 3,
                                           'Maj2': 2,  
                                           'Sev': 1,
                                           'Sal': 0})

In [29]:
ames['Fireplace_Qu'].value_counts()

No_Fireplace    1422
Gd               744
TA               600
Fa                75
Po                46
Ex                43
Name: Fireplace_Qu, dtype: int64

In [30]:
ames['Fireplace_Qu'] = ames['Fireplace_Qu'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1,
                                           'No_Fireplace': 0})

In [31]:
ames['Garage_Qual'].value_counts()

TA           2615
No_Garage     159
Fa            124
Gd             24
Po              5
Ex              3
Name: Garage_Qual, dtype: int64

In [32]:
ames['Garage_Qual'] = ames['Garage_Qual'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1,
                                           'No_Garage': 0})

In [33]:
ames['Garage_Cond'].value_counts()

TA           2665
No_Garage     159
Fa             74
Gd             15
Po             14
Ex              3
Name: Garage_Cond, dtype: int64

In [34]:
ames['Garage_Cond'] = ames['Garage_Cond'].map({'Ex': 5,
                                           'Gd': 4,
                                           'TA': 3,
                                           'Fa': 2,
                                           'Po': 1,
                                           'No_Garage': 0})

In [35]:
ames['Pool_QC'].value_counts()

No_Pool    2917
Gd            4
Ex            4
TA            3
Fa            2
Name: Pool_QC, dtype: int64

In [36]:
ames['Pool_QC'] = ames['Pool_QC'].map({'Ex': 4,
                                           'Gd': 3,
                                           'TA': 2,
                                           'Fa': 1,
                                           'No_Pool': 0})

In [37]:
ames['Garage_Finish'].value_counts()

Unf          1231
RFn           812
Fin           728
No_Garage     159
Name: Garage_Finish, dtype: int64

In [38]:
ames['Garage_Finish'] = ames['Garage_Finish'].map({'Fin': 3,
                                           'RFn': 2,
                                           'Unf': 1,
                                           'No_Garage': 0})

#### Doing Feature Engineering to combine some columns and dropping them later

In [39]:
ames['Garage_Cars_x_Area'] = ames['Garage_Cars']*ames['Garage_Area']
ames['Total_SF'] = ames['Total_Bsmt_SF'] + ames['1st_Flr_SF'] + ames['2nd_Flr_SF']
ames.drop(columns=['Total_Bsmt_SF', '1st_Flr_SF', '2nd_Flr_SF', 'Garage_Cars', 'Garage_Area'], inplace=True)

#### Doing Polynomial Features for the 5 stronges correlation features with degree = 3

In [40]:
poly = PolynomialFeatures(degree=3, include_bias=False)
P0 = ames[['Overall_Qual', 'Exter_Qual', 'Gr_Liv_Area', 'Kitchen_Qual','Total_SF']]
features = list(P0.columns)
P_poly = poly.fit_transform(P0)
P1 = pd.DataFrame(P_poly, columns=poly.get_feature_names(features))
P = P0.merge(P1, left_index=True, right_index=True)

#### Merging our Polynomial Features DataFrame with our original DataFrame

In [41]:
ames = ames.merge(P, left_index=True, right_index=True)

#### Getting dummies for all categorical columns

In [42]:
ames = pd.get_dummies(ames, drop_first=True)

#### Splitting back into train and test data

In [43]:
train_a = ames[:2051]

In [44]:
test_a = ames[2051:]

#### Adding back the SalePrice to the train data

In [45]:
train_a['SalePrice'] = ames1['SalePrice']

/Users/jurgenarias/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Outliers

I identified outliers with stats, slicing and box plots. I planned to keep them thinking that the log transformation would take care of them but the log transformation did not help our model so I did not use it. I dropped two houses with unusually large square footage and really low price. 

In [46]:
train_a.loc[((train_a['Total_SF']>10_000))]

,Id,PID,MS_SubClass,Lot_Frontage,Lot_Area,Overall_Qual,Overall_Cond,Year_Built,Year_Remod/Add,Mas_Vnr_Area,...,Sale_Type_CWD,Sale_Type_Con,Sale_Type_ConLD,Sale_Type_ConLI,Sale_Type_ConLw,Sale_Type_New,Sale_Type_Oth,Sale_Type_VWD,Sale_Type_WD,SalePrice
960,1499,908154235,60,313.0,63887,10,5,2008,2008,796.0,...,0,0,0,0,0,1,0,0,0,160000
1885,2181,908154195,20,128.0,39290,10,5,2008,2009,1224.0,...,0,0,0,0,0,1,0,0,0,183850


In [47]:
train_a.loc[((train_a['Total_SF']>10_000))]['SalePrice']

960     160000
1885    183850
Name: SalePrice, dtype: int64

In [48]:
train_a.drop([960, 1885], axis=0, inplace=True)

/Users/jurgenarias/anaconda3/envs/dsi/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Checking the correlations

In [49]:
train_a.corr().abs()[['SalePrice']].sort_values(by=['SalePrice'],ascending=False).head(6)

,SalePrice
SalePrice,1.000000
Overall_Qual Kitchen_Qual Total_SF,0.929374
Overall_Qual Exter_Qual Total_SF,0.925073
Overall_Qual Total_SF,0.921189
Overall_Qual^2 Total_SF,0.920266
Exter_Qual Kitchen_Qual Total_SF,0.911415


In [50]:
train_a.to_csv('../datasets/train_a.csv', index=False)

In [51]:
test_a.to_csv('../datasets/test_a.csv', index=False)